# Run ROH Detection

In [104]:
import socket as socket
import os as os
import sys as sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("bionc"):
    print("Leipzig Cluster detected!")
    path = "/mnt/archgen/users/hringbauer/git/ibd_gurgy/"
    #sys.path.append("/mnt/archgen/users/hringbauer/git/ibd_gurgy/") 
    
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd())

bionc21
Leipzig Cluster detected!
/mnt/archgen/users/hringbauer/git/ibd_gurgy


In [39]:
df = pd.read_csv("./data/roh/combined_ROH_v1.tsv", sep='\t')

# Rename IIDs to remove ugly library name
#iids = df["iid"].str.split(".").values
#iids = [i[0] for i in iids]
#df["iid"] = iids

In [121]:
df2 = pd.read_csv("/mnt/archgen/users/rivollat/Interact/Neo/GRG/20210215_Datafreeze_1240K/genotypes/TF/France_MN_GRG_all_20210215.TF.allChrom.ind", sep="\t", header=None)
df2.columns = ["iid", "sex", "clst"]

df21 = pd.read_csv("/mnt/archgen/users/rivollat/Interact/Neo/GRG/20210215_Datafreeze_1240K/genotypes/TF/France_MN_GRG_all_20210215.TF.allChrom.cov.txt", 
                  sep="\t")

#idx = df21["called SNPs"] > 3e5
#np.sum(idx)
df2["n_cov_snps"] = df21["called SNPs"]
dft = df2[df2["n_cov_snps"]>3e5]
print(f"Loaded {len(dft)} IIDs with sufficiently many SNPs")
iids = dft["iid"].values
dft.to_csv("/mnt/archgen/users/hringbauer/git/ibd_gurgy/data/meta/roh_meta.v1.csv", sep=",", index=False)

Loaded 86 IIDs with sufficiently many SNPs


# Run hapROH

In [61]:
from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind 

In [ ]:
%%time
path_targets = "/mnt/archgen/users/rivollat/Interact/Neo/GRG/20210215_Datafreeze_1240K/genotypes/TF/France_MN_GRG_all_20210215.TF.allChrom"

hapsb_ind(iid=iids[0], chs=range(1, 2), 
          path_targets=path_targets, # The path before the .ind, .snp, .geno
          h5_path1000g='/mnt/archgen/users/hringbauer/data/hapROH.globalRef/chr', 
          meta_path_ref='/mnt/archgen/users/hringbauer/data/hapROH.globalRef/meta_df_all.csv', 
          folder_out="/mnt/archgen/users/hringbauer/git/ibd_gurgy/output/roh/",  # Folder where you want to save the results to 
          processes=1, output=True, roh_min_l_final=0.04,
          readcounts=False, logfile=False, combine=True)

# Run Jobs on cluster

In [105]:
!cd /mnt/archgen/users/hringbauer/git/ibd_gurgy/cluster/roh/; qsub ./call_roh.qsub  ### Submit Job

Your job-array 3032847.1-86:1 ("call_ROH_Gurgy") has been submitted


In [112]:
!qstat -u harald_ringbauer ### Check what jobs are running

# Process ROH

In [113]:
from hapsburg.PackagesSupport.pp_individual_roh_csvs import pp_individual_roh

In [122]:
%%time
### Postprocess the two Individuals from above and combine into one results .csv

df1 = pp_individual_roh(iids, meta_path="/mnt/archgen/users/hringbauer/git/ibd_gurgy/data/meta/roh_meta.v1.csv", 
                        base_folder="/mnt/archgen/users/hringbauer/git/ibd_gurgy/output/roh/",
                        save_path="/mnt/archgen/users/hringbauer/git/ibd_gurgy/output/roh/combined_roh05.csv", 
                        output=False, min_cm=[4, 8, 12, 20], snp_cm=50, 
                        gap=0.5, min_len1=2.0, min_len2=4.0)

Loaded 86 / 86 Individuals from Meta
Saved to: /mnt/archgen/users/hringbauer/git/ibd_gurgy/output/roh/combined_roh05.csv
CPU times: user 1.96 s, sys: 16.9 ms, total: 1.97 s
Wall time: 2.23 s
